# TV Script Generation

In this project, I am going to generate my own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  I will be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The neural network that I will build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

In [5]:
import numpy as np
from collections import Counter

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import helper

In [1]:
# load in data
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

In [9]:
temp_text = text.split('\n')
temp_text = ' '.join(temp_text)
words = temp_text.split()
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab)}

In [10]:
print(vocab_to_int)

{'the': 0, 'i': 1, 'you': 2, 'jerry:': 3, 'a': 4, 'to': 5, 'george:': 6, 'elaine:': 7, 'kramer:': 8, 'and': 9, 'of': 10, 'in': 11, 'what': 12, 'is': 13, 'it': 14, 'that': 15, 'this': 16, 'my': 17, "i'm": 18, "it's": 19, 'have': 20, 'with': 21, 'on': 22, 'for': 23, 'he': 24, "don't": 25, 'was': 26, 'do': 27, 'just': 28, 'like': 29, 'me': 30, 'are': 31, 'get': 32, 'not': 33, 'your': 34, 'all': 35, 'well,': 36, 'oh,': 37, 'know': 38, 'we': 39, 'so': 40, 'be': 41, 'at': 42, 'got': 43, 'about': 44, "you're": 45, "that's": 46, 'yeah,': 47, 'out': 48, 'they': 49, 'his': 50, 'she': 51, 'how': 52, 'no,': 53, 'think': 54, 'it.': 55, 'if': 56, 'no': 57, 'up': 58, 'but': 59, 'can': 60, 'her': 61, 'see': 62, 'go': 63, 'gonna': 64, 'going': 65, 'did': 66, 'want': 67, "he's": 68, 'come': 69, 'yeah.': 70, 'hey,': 71, 'uh,': 72, "can't": 73, 'why': 74, "i'll": 75, 'one': 76, 'know,': 77, 'from': 78, 'oh': 79, 'an': 80, 'well': 81, 'tell': 82, 'you.': 83, 'me.': 84, 'could': 85, 'what?': 86, 'really': 8

In [11]:
int_to_vocab = {ii:word for word,ii in vocab_to_int.items()}
print(int_to_vocab)

{0: 'the', 1: 'i', 2: 'you', 3: 'jerry:', 4: 'a', 5: 'to', 6: 'george:', 7: 'elaine:', 8: 'kramer:', 9: 'and', 10: 'of', 11: 'in', 12: 'what', 13: 'is', 14: 'it', 15: 'that', 16: 'this', 17: 'my', 18: "i'm", 19: "it's", 20: 'have', 21: 'with', 22: 'on', 23: 'for', 24: 'he', 25: "don't", 26: 'was', 27: 'do', 28: 'just', 29: 'like', 30: 'me', 31: 'are', 32: 'get', 33: 'not', 34: 'your', 35: 'all', 36: 'well,', 37: 'oh,', 38: 'know', 39: 'we', 40: 'so', 41: 'be', 42: 'at', 43: 'got', 44: 'about', 45: "you're", 46: "that's", 47: 'yeah,', 48: 'out', 49: 'they', 50: 'his', 51: 'she', 52: 'how', 53: 'no,', 54: 'think', 55: 'it.', 56: 'if', 57: 'no', 58: 'up', 59: 'but', 60: 'can', 61: 'her', 62: 'see', 63: 'go', 64: 'gonna', 65: 'going', 66: 'did', 67: 'want', 68: "he's", 69: 'come', 70: 'yeah.', 71: 'hey,', 72: 'uh,', 73: "can't", 74: 'why', 75: "i'll", 76: 'one', 77: 'know,', 78: 'from', 79: 'oh', 80: 'an', 81: 'well', 82: 'tell', 83: 'you.', 84: 'me.', 85: 'could', 86: 'what?', 87: 'really

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing. We need to implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, we need to transform the words to ids.  In this function, I will create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Then I will return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [1]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    #count words
    counts = Counter(text)
    
    #sort words by count
    vocab = sorted(counts, key=counts.get, reverse=True)
    
    #create dictionary with unique words as keys, counts as values
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
    
    #now invert the key, value mapping 
    int_to_vocab = {ii:word for word, ii in vocab_to_int.items()}

    # return tuple
    return (vocab_to_int, int_to_vocab)

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Thus, we can implement a function `token_lookup` to return a dictionary that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  This is a dictionary where the following symbols are the keys and the values are the tokens:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word.

In [2]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    token_dict = {'.':'<PERIOD>',
                  ',':'<COMMA>',
                  '"':'<QUOTATION>',
                  ';':'<SEMICOLON>',
                  '!':'<EXCLAMATION>',
                  '?':'<QUESTION>',
                  '(':'<LEFT_PARENTHESES>',
                  ')':'<RIGHT_PARENTHESES>',
                  '-':'<DASH>',
                  '\n':'<RETURN>',}
        
    return token_dict

## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file.

In [14]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

## Checkpoint 1

In [15]:
#create a checkpoint so that if we decide to come back to the notebook we can start here
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, I built the components of an RNN by implementing the RNN Module and forward and backpropagation functions.

In [16]:
#check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

We can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader like so:
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
We need to implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, we will need to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [17]:
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    #caclulate number of full batches in data
    total_batches = len(words) // (batch_size)
    
    #only keep enough words to make full batches
    words = words[:total_batches*batch_size]
    
    #create empty lists to append to
    X, Y = [], []
    
    #iterate over words
    #need to subtract_sequence_length to prevent out of bounds errors
    for i in range(0, len(words)-sequence_length):
        x_batch = words[i:sequence_length+i]
        y_batch = words[sequence_length+i]
        
        #append batches to lists
        X.append(x_batch)
        Y.append(y_batch)
    
    #convert lists to numpy arrays before turning into torch tensors
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    #create feature tensors to be used by dataloader
    feature_tensor = torch.from_numpy(X)
    target_tensor = torch.from_numpy(Y)
    
    #create torch dataset and dataloader
    training_data = TensorDataset(feature_tensor, target_tensor)
    dataloader = DataLoader(training_data, shuffle=False, batch_size=batch_size)
    
    # return a dataloader
    return dataloader

### Testing the dataloader 

Below, we can generate some test text data and defining a dataloader using the function that was defined, above. Then, we can obtain batches of inputs `sample_x` and targets `sample_y` from the dataloader.

This code should return something like the following (likely in a different order, depending on if the data was shuffled or not):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values
Also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [18]:
# test dataloader
test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


---
## Build the Neural Network
We can now implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module) using the LSTM class.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

In [6]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        self.drop = nn.Dropout(0.3)
        self.fc1 = nn.Linear(hidden_dim, output_size)    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """

        # return one batch of output word scores and the hidden state
        #get batch size from input
        batch_size = nn_input.size(0)
        
        #downsize vocab to embedding dimension
        embedding = self.embedding(nn_input)
        
        #pass word embedding to lstm layer, flatten before passing to dropout layer
        lstm_out, hidden = self.lstm(embedding, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
    
        #pass through dropout and linear layers
        x = self.fc1(lstm_out)
        
        # reshape into (batch_size, seq_length, output_size)
        x = x.view(batch_size, -1, self.output_size)
        
        #get just the last output for most likely next word
        x = x[:, -1]
        
        
        return x, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

### Define forward and backpropagation

Once we have the RNN architecture defined, we can apply forward and back propagation through the function that we create below. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`.

In [7]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    #maximum gradient value to clip at (to prevent exploding gradients)
    clip = 5
    
    # move data, model to GPU if available
    if(torch.cuda.is_available()):
        rnn.cuda()
        x, y = inp.cuda(), target.cuda()
    
     #create new variables for the hidden state
    h = tuple(each.data for each in hidden)
    
    #zero out previous gradients
    rnn.zero_grad()
    
    #pass input and current hidden state to network
    output, h = rnn(x, h)
    
    # perform backpropagation and optimization
    #calculate the loss
    loss = criterion(output, y)
    
    #backpropagate
    loss.backward()
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    
    #take a step towards optimum
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), h

## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter.

In [21]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [23]:
# Sequence Length
sequence_length = 10  # of words in a sequence

# Batch Size
batch_size = 256

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [24]:
# Number of Epochs
num_epochs = 20

# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)

# Output size
output_size = vocab_size

# Embedding Dimension
embedding_dim = 128

# Hidden Dimension
hidden_dim = 128

# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
#this will print 1 time per epoch
show_every_n_batches = 2000

In [25]:
from workspace_utils import active_session
with active_session():
    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('./save/trained_rnn', trained_rnn)
    print('Model Trained and Saved')

Training for 20 epoch(s)...
Epoch:    1/20    Loss: 5.052135448694229

Epoch:    2/20    Loss: 4.368847068616312

Epoch:    3/20    Loss: 4.073917078143593

Epoch:    4/20    Loss: 3.919697717384056

Epoch:    5/20    Loss: 3.8128965117279017

Epoch:    6/20    Loss: 3.734421539744734

Epoch:    7/20    Loss: 3.67207032272674

Epoch:    8/20    Loss: 3.6214772786280904

Epoch:    9/20    Loss: 3.5778024232219834

Epoch:   10/20    Loss: 3.5486242137966983

Epoch:   11/20    Loss: 3.5146502474413293

Epoch:   12/20    Loss: 3.4806169551095847

Epoch:   13/20    Loss: 3.4565724501690576

Epoch:   14/20    Loss: 3.4337506637195387

Epoch:   15/20    Loss: 3.4128253379841493

Epoch:   16/20    Loss: 3.395347772095007

Epoch:   17/20    Loss: 3.3786647676429014

Epoch:   18/20    Loss: 3.362857057964052

Epoch:   19/20    Loss: 3.3481411953065505

Epoch:   20/20    Loss: 3.330579508524875



/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Deciding on your model hyperparameters

- `n_layers`: 2-3 layers is provided as best choice by Andrej Karpathy for advice on how to train RNNs: https://github.com/karpathy/char-rnn#tips-and-tricks 
- `batch_size`: This was the largest value for what would fit on GPU. I tried larger and smaller values first, but only powers of 2. 256 was also the value that sped up training the most for me. Smaller batch sizes took longer to train for some reason.
- `epochs`: I knew I would need to train for at least a few epochs, but each epoch ended up taking a long time. It would have been smarter to time one epoch, see how it performed, and then have a better estimate of how many total epochs I would need. I tried 10 epochs, but it barely missed getting the desired loss value of 3.5. So I decided to double the epochs to be sure I would reach it.
- `embedding_dim, hidden_dim`: I tried values based on powers of 2, I have heard that this allows for faster training.
- `sequence_length`: I left this value at 10 (the value I used for testing my batch_data() function), but I could have tried other values. I took this to be a good intermediate value. At a smaller value, I'm not sure the model would have picked up on the context of the predicted word, but for larger values I imagine there is either a diminishing returns scenario (where the value of adding a longer sequence drops off at some point) or the compute needed to add additional words to the sequence length becomes extremely compute intensive.

Originally, I also had a Dropout layer between the LSTM flatten layer and the Linear layer. However, the LSTM layer has a dropout attribute, and so I wasn't sure if it was necessary. By the time I had tuned all the other parameters it didn't seem like I needed it to improve the loss.

## Checkpoint 2

After running the above training cell, the model will be saved by name, `trained_rnn`, and if we save theournotebook progress, **we can pause here and come back to this code at another time**. Thus we can resume our progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [7]:
#check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

#this needs to be included again if we are loading from a checkpoint
sequence_length = 10

#for this to actually work, you need to run the
#train_on_gpu and RNN class cells again, as well as import numpy
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, we can use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. We will use the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [26]:
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [27]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word]-1, gen_length)
print(generated_script)

jerry: oh. no. i don't think so.

jerry: oh, no no no, i'm not gonna be able to 18 of you and get the money.

elaine: i can't find that.

jerry: i know, but i can tell you that i was going to do this, but it is a one--

jerry:(pointing at him) well, i guess it's a good time.

jerry: oh no.

jerry: i don't know how you can do this!

george: what are you doing here?

elaine: i don't think it's so much as this is a surface of the taxpayers'.(laughs)

jerry: hey!!

george: hey! hey, i got some money to make the catsup, and i have to tell you that, but i don't have any trouble controlling. i got the one.

jerry: oh no!!!...

jerry:(to jerry) i can't believe it.

jerry: so, it's not going to get out of the endless.(laughs)

jerry: hey!

kramer: hey, hey!

elaine: oh, hi.

elaine: what are we doing here?

jerry:(to the phone) hey, hey, hey, jerry!

george: oh! oh. hey, what are you gonna do to eat?!

jerry: what?

elaine: no.

jerry:(to jerry) hey, what is it?

jerry: oh, it's a moops, huh?(j

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [28]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 